# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
import pymagicc
from pymagicc.io import MAGICCData
import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
TEST_DATA_PATH = join("..", "tests", "test_data")

## Reading in a CEDS csv

To read in CEDS csv's, we make use of the `pyam` library which is specifically designed for this purpose.

In [4]:
def read_ceds_csv(file_to_read):
    return pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )

ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
ceds_pyam_df  # this just shows the type of ceds_pyam_df
ceds_pyam_df.data  # this returns the underlying DataFrame

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


,model,scenario,region,variable,unit,year,value
1138,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2015,2
1229,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2015,5
1312,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2015,9
1395,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2015,2
1486,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC,Mt BC/yr,2015,10
1577,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,10
3414,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2020,7
3505,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2020,9
3588,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2020,9
3671,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2020,8


## Filtering an `IamDataFrame`

The `pyam` library provides some very natural ways of filtering their DataFrames. We repeat the most relevant of these here, please see the pyam docs (TODO: add link) for more detail.

### Filtering to check variable aggregation

Firstly we show how to filter the data table to check that the sum of a given variable's sub-categories is equal to its declared total.

We start by extracting a single model-scenario-region-variable combination (where 'variable' here means the emissions for a given variable in all of its different categories).

In [5]:
# start by cutting our data table to a single 
# model-scenario-region combination
msrv_df = ceds_pyam_df.filter(
    variable='Emissions|BC*',
    model="MODEL-NAME-HYPHENS",
    scenario="SCENARIO-NAME-HYPHENS",
    region="R5ASIA",
)

msrv_df.models()
msrv_df.scenarios()
msrv_df.regions()
msrv_df.variables()
msrv_df.data

0    MODEL-NAME-HYPHENS
Name: model, dtype: object

0    SCENARIO-NAME-HYPHENS
Name: scenario, dtype: object

0    R5ASIA
Name: region, dtype: object

0                                 Emissions|BC
1      Emissions|BC|Agricultural Waste Burning
2                   Emissions|BC|Energy Sector
3                  Emissions|BC|Forest Burning
4               Emissions|BC|Grassland Burning
5               Emissions|BC|Industrial Sector
6                    Emissions|BC|Peat Burning
7    Emissions|BC|Residential Commercial Other
8           Emissions|BC|Transportation Sector
9                           Emissions|BC|Waste
Name: variable, dtype: object

,model,scenario,region,variable,unit,year,value
0,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2015,46
2276,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2020,24
4552,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2030,40
6828,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2040,49
9104,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2050,45
11380,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2060,35
13656,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2070,52
15932,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2080,48
18208,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2090,51
20484,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2100,43


Now we want to sum all of the subcategories for the variable and check if they sum to the total.

First we create a data frame which only contains the emissions subcategories.

In [6]:
# next filter our data to only get subcategores
msrv_subcat_df = msrv_df.filter(
    variable='Emissions|BC|*',
)
msrv_subcat_df.variables()  # notice that Emissions|BC is no longer there
msrv_subcat_df.data

0      Emissions|BC|Agricultural Waste Burning
1                   Emissions|BC|Energy Sector
2                  Emissions|BC|Forest Burning
3               Emissions|BC|Grassland Burning
4               Emissions|BC|Industrial Sector
5                    Emissions|BC|Peat Burning
6    Emissions|BC|Residential Commercial Other
7           Emissions|BC|Transportation Sector
8                           Emissions|BC|Waste
Name: variable, dtype: object

,model,scenario,region,variable,unit,year,value
1,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2015,10
2277,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2020,6
4553,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2030,7
6829,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2040,10
9105,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2050,4
11381,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2060,4
13657,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2070,1
15933,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2080,5
18209,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2090,7
20485,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC|Agricultural Waste Burning,Mt BC/yr,2100,0


Then we use `pivot_table` method from `pyam` to sum to a variable total.

In [7]:
msrv_subcat_sum_df = msrv_subcat_df.pivot_table(
    index=['year'], 
    columns=['scenario', 'model', 'region', 'unit'], 
    values='value',
    aggfunc='sum',  # as variable is not mentioned above, it will be summed over
)
msrv_subcat_sum_df

scenario,SCENARIO-NAME-HYPHENS
model,MODEL-NAME-HYPHENS
region,R5ASIA
unit,Mt BC/yr
year,
2015,46
2020,24
2030,40
2040,49
2050,45
2060,35


We can now compare this total to the declared total and see if they match. We use the pivot function again to ensure that our indexes make sense for the comparison.

In [8]:
msrv_total_df = msrv_df.filter(
    variable='Emissions|BC',
).pivot_table(
    index=['year'], 
    columns=['scenario', 'model', 'region', 'unit'], 
    values='value',
    aggfunc='sum', 
)
msrv_total_df

scenario,SCENARIO-NAME-HYPHENS
model,MODEL-NAME-HYPHENS
region,R5ASIA
unit,Mt BC/yr
year,
2015,46
2020,24
2030,40
2040,49
2050,45
2060,35


In [9]:
msrv_subcat_sum_df.eq(msrv_total_df.values).all()
msrv_subcat_sum_df.eq(msrv_total_df.values)

scenario               model               region  unit    
SCENARIO-NAME-HYPHENS  MODEL-NAME-HYPHENS  R5ASIA  Mt BC/yr    True
dtype: bool

scenario,SCENARIO-NAME-HYPHENS
model,MODEL-NAME-HYPHENS
region,R5ASIA
unit,Mt BC/yr
year,
2015,True
2020,True
2030,True
2040,True
2050,True
2060,True


We can now put all of this into a single class which will loop over all the models, scenarios, regions and variables and check if the sectoral sum is equal to the declared total. As our data is random, we expect that all of the sums will be incorrect except for the one we just tried (which we adjusted by hand).

In [13]:
class CedsDb(object):
    def __init__(self, pyam_df):
        self.pyam_df = pyam_df
        self._model_scen_pyam_df = None
    
    def check_sectoral_sums(self):
        for (model, scenario), df in self.pyam_df.data.groupby(["model", "scenario"]):
            self._check_sectoral_sums_model_scenario(model, scenario)
            
    def _check_sectoral_sums_model_scenario(self, model, scenario):
        self._model_scen_pyam_df = self.pyam_df.filter(
            model=model,
            scenario=scenario,
        )
        variables = self._model_scen_pyam_df.filter(
            level=1
        ).variables()
        for variable in variables:
            self._check_sectoral_sums_variable(variable)
            
    def _check_sectoral_sums_variable(self, variable):
        regions = self._model_scen_pyam_df.filter(
            variable=variable,
        ).regions()
        for region in regions:
            self._check_sectoral_sums_variable_region(variable, region)
    
    def _check_sectoral_sums_variable_region(self, variable, region):
        total_df = self._get_total_df_check_sectoral_sums_variable_region(variable, region)
        
        model = total_df.columns.get_level_values("model")[0]
        scenario = total_df.columns.get_level_values("scenario")[0]

        sector_sum_df = self._get_sector_sum_df_check_sectoral_sums_variable_region(variable, region)
        if sector_sum_df.empty:
            print("No sectoral data for {}, {}, {}, {}".format(model, scenario, variable, region))
        elif sector_sum_df.eq(total_df).all().all():
            print("All good for {}, {}, {}, {}".format(model, scenario, variable, region))
        else:
            print("Sector sum does not match total for {}, {}, {}, {}".format(model, scenario, variable, region))
            
    def _get_total_df_check_sectoral_sums_variable_region(self, variable, region):
        return self._model_scen_pyam_df.filter(
            variable=variable,
            region=region,
        ).pivot_table(
            index=['year'], 
            columns=['scenario', 'model', 'region', 'unit'], 
            values='value',
            aggfunc='sum', 
        )
    
    def _get_sector_sum_df_check_sectoral_sums_variable_region(self, variable, region):
        return self._model_scen_pyam_df.filter(
            variable='{}|*'.format(variable),
            region=region,
        ).pivot_table(
            index=['year'], 
            columns=['scenario', 'model', 'region', 'unit'], 
            values='value',
            aggfunc='sum', 
        )
    
ceds_db = CedsDb(ceds_pyam_df)

ceds_db.check_sectoral_sums()

Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, R5ASIA
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, R5LAM
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, R5MAF


Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, R5OECD
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, R5REF
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|BC, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|C2F6, World


No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CCl4, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CF4, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CFC-11, World


No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CFC-113, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CFC-114, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CFC-115, World


No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CFC-12, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH3, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH3CCl3, World
No sectoral data for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH3Cl, World
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, R5ASIA
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, R5LAM
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, R5MAF
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, R5OECD
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, R5REF
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emissions|CH4, World
Sector sum does not match total for MODEL-NAME-HYPHENS, SCENARIO-A-B-CDE-2, Emi

...

No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH3CCl3, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH3Cl, World
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, R5ASIA
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, R5LAM
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, R5MAF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, R5OECD
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, R5REF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CH4, World
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CO, R5ASIA
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|CO, R5LAM
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|

No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|HCFC-141b, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|HCFC-142b, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|HCFC-22, World


No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|HFC, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Halon1202, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Halon1211, World


No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Halon1301, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Halon2402, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|N2O, World
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, R5ASIA
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, R5LAM
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, R5MAF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, R5OECD
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, R5REF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NH3, World
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NOx, R5ASIA
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|NOx, R5

Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|OC, R5OECD
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|OC, R5REF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|OC, World
No sectoral data for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|SF6, World
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, R5ASIA
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, R5LAM
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, R5MAF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, R5OECD
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, R5REF
Sector sum does not match total for NAME-MODEL-2, SCENARIO-NAME-HYPHENS, Emissions|Sulfur, World
Sector sum does not match total for NAME-MODEL-2, SCENARI

### Filtering to check regional aggregation

In [ ]:
for label, df in ceds_pyam_df.groupby(axis=1, level=groupby_and_drop_levels):
    metadata = {level: a[i] for i, level in enumerate(groupby_and_drop_levels)}
    metadata
    b
    b.columns = b.columns.droplevel(level=groupby_and_drop_levels)
    b

## Reshaping an `IamDataFrame`

Lastly we show how to reshape an `IamDataFrame` to get it into the format expected by `MAGICCData` so we can then write files with the data in it.

In [ ]:
def reshape_pyam_df(pyam_df):
    raw_df = pyam_df.data
    
    reindexed_df = raw_df.set_index(
        ["model", "scenario", "region", "variable", "unit", "year"]
    ).unstack().T
    reindexed_df.index = reindexed_df.index.get_level_values("year")
    reindexed_df.index.name = "YEAR"
    models = reindexed_df.columns.get_level_values("model")
    scenarios = reindexed_df.columns.get_level_values("scenario")
    regions = reindexed_df.columns.get_level_values("region")
    variables = reindexed_df.columns.get_level_values("variable")
    units = reindexed_df.columns.get_level_values("unit")
    todos = ["SET"] * len(units)
    reindexed_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    return reindexed_df

In [ ]:
ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
ceds_pyam_df.regions()

In [ ]:
# get the sum of all variable subcategories for a variable
filtered_table = ceds_pyam_df.filter(variable='Emissions|BC|*')
filtered_table.variables()
filtered_table.data

In [ ]:
# create a pivot table from the filtered table
# where variables have been summed
filtered_table.pivot_table(
    index=['year'], 
    columns=['scenario', 'model', 'region', 'unit'], 
    values='value',
    aggfunc='sum',
)

In [ ]:
# get the sum of a variable for all regions
filtered_table = ceds_pyam_df.filter(variable='Emissions|BC')
filtered_table.variables()
filtered_table.data

In [ ]:
# get the sum of a variable for all regions
filtered_table = ceds_pyam_df.filter(region='World')
filtered_table.variables()
filtered_table.data

In [ ]:
# create a pivot table from the filtered table
# where regions have been summed
filtered_table.pivot_table(
    index=['year'], 
    columns=['scenario', 'model', 'variable', 'unit'], 
    values='value',
    aggfunc='sum',
)

In [ ]:
ceds_pymagicc_df = reshape_pyam_df(ceds_pyam_df)
groupby_and_drop_levels = ["MODEL", "SCENARIO"]
for a, b in ceds_pymagicc_df.groupby(axis=1, level=groupby_and_drop_levels):
    metadata = {level: a[i] for i, level in enumerate(groupby_and_drop_levels)}
    metadata
    b
    b.columns = b.columns.droplevel(level=groupby_and_drop_levels)
    b